<a href="https://colab.research.google.com/github/Aasthapriy44/AI-Lab/blob/main/Week3_Misplacedtiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import heapq


class Node:
    def __init__(self, state, parent=None, action=None, cost=0, heuristic=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.cost = cost
        self.heuristic = heuristic
        self.total_cost = cost + heuristic

    def __lt__(self, other):
        return self.total_cost < other.total_cost


def find_blank(state):
    for i in range(3):
        for j in range(3):
            if state[i][j] == 0:
                return (i, j)


def get_neighbors(state):
    row, col = find_blank(state)
    neighbors = []

    if row > 0:
        new_state = [list(row) for row in state]
        new_state[row][col], new_state[row - 1][col] = new_state[row - 1][col], new_state[row][col]
        neighbors.append(('up', [list(row) for row in new_state]))
    if row < 2:
        new_state = [list(row) for row in state]
        new_state[row][col], new_state[row + 1][col] = new_state[row + 1][col], new_state[row][col]
        neighbors.append(('down', [list(row) for row in new_state]))
    if col > 0:
        new_state = [list(row) for row in state]
        new_state[row][col], new_state[row][col - 1] = new_state[row][col - 1], new_state[row][col]
        neighbors.append(('left', [list(row) for row in new_state]))
    if col < 2:
        new_state = [list(row) for row in state]
        new_state[row][col], new_state[row][col + 1] = new_state[row][col + 1], new_state[row][col]
        neighbors.append(('right', [list(row) for row in new_state]))

    return neighbors


def misplaced_tiles(state):
    goal_state = [[1, 2, 3], [8, 0, 4], [7, 6, 5]]
    misplaced_count = 0
    for i in range(3):
        for j in range(3):
            if state[i][j] != goal_state[i][j] and state[i][j] != 0:
                misplaced_count += 1
    return misplaced_count


def solve_8_puzzle(initial_state):
    goal_state = [[1, 2, 3], [8, 0, 4], [7, 6, 5]]
    priority_queue = []
    heapq.heappush(priority_queue, Node(initial_state, heuristic=misplaced_tiles(initial_state)))
    visited = set()
    visited.add(tuple(map(tuple, initial_state)))

    while priority_queue:
        current_node = heapq.heappop(priority_queue)

        if current_node.state == goal_state:
            path = []
            while current_node:
                path.append((current_node.action, current_node.state))
                current_node = current_node.parent
            return path[::-1]

        for action, neighbor_state in get_neighbors(current_node.state):
            if tuple(map(tuple, neighbor_state)) not in visited:
                new_node = Node(neighbor_state, current_node, action,
                               current_node.cost + 1, misplaced_tiles(neighbor_state))
                heapq.heappush(priority_queue, new_node)
                visited.add(tuple(map(tuple, neighbor_state)))

    return None  # No solution found


if __name__ == "__main__":
    initial_state = [[2, 8, 3], [1, 6, 4], [0, 7, 5]]
    solution = solve_8_puzzle(initial_state)

    if solution:
        print("Solution found:")
        for action, state in solution:
            print(f"Action: {action}")
            for row in state:
                print(row)
            print()
    else:
        print("No solution found.")

Solution found:
Action: None
[2, 8, 3]
[1, 6, 4]
[0, 7, 5]

Action: right
[2, 8, 3]
[1, 6, 4]
[7, 0, 5]

Action: up
[2, 8, 3]
[1, 0, 4]
[7, 6, 5]

Action: up
[2, 0, 3]
[1, 8, 4]
[7, 6, 5]

Action: left
[0, 2, 3]
[1, 8, 4]
[7, 6, 5]

Action: down
[1, 2, 3]
[0, 8, 4]
[7, 6, 5]

Action: right
[1, 2, 3]
[8, 0, 4]
[7, 6, 5]



In [11]:
import heapq

class PuzzleState:
    def __init__(self, board, parent, move, depth, cost):
        self.board = board
        self.parent = parent
        self.move = move
        self.depth = depth
        self.cost = cost

    def __lt__(self, other):
        return self.cost < other.cost

def misplaced_tiles(board, goal):
    return sum(1 for i in range(9) if board[i] != goal[i] and board[i] != 0)

def a_star_search(initial_state, goal_state):
    explored = set()
    priority_queue = []
    initial = PuzzleState(initial_state, None, None, 0, misplaced_tiles(initial_state, goal_state))
    heapq.heappush(priority_queue, initial)

    while priority_queue:
        current_state = heapq.heappop(priority_queue)
        explored.add(tuple(current_state.board))

        if current_state.board == goal_state:
            return current_state

        for move in possible_moves(current_state.board):
            new_board = apply_move(current_state.board, move)
            if tuple(new_board) not in explored:
                g_n = current_state.depth + 1
                h_n = misplaced_tiles(new_board, goal_state)
                new_state = PuzzleState(new_board, current_state, move, g_n, g_n + h_n)
                heapq.heappush(priority_queue, new_state)

    return None

def possible_moves(board):

    moves = []
    empty_index = board.index(0)
    if empty_index % 3 > 0:  # Move left
        moves.append(-1)
    if empty_index % 3 < 2:  # Move right
        moves.append(1)
    if empty_index > 2:  # Move up
        moves.append(-3)
    if empty_index < 6:  # Move down
        moves.append(3)
    return moves

def apply_move(board, move):

    new_board = board[:]
    empty_index = new_board.index(0)
    new_index = empty_index + move
    new_board[empty_index], new_board[new_index] = new_board[new_index], new_board[empty_index]
    return new_board

def move_to_string(move):

    move_dict = {
        -1: "Left",
        1: "Right",
        -3: "Up",
        3: "Down"
    }
    return move_dict.get(move, "Start")  # Default to "Start" if move is None

def print_solution(state):

    path = []
    while state:
        path.append(state)
        state = state.parent

    # Reverse the path to print from initial state to goal
    path.reverse()
    print("Solution path:")
    for step in path:
        move_str = move_to_string(step.move)  # Get string representation of the move
        g_n = step.depth  # Depth (g(n))
        h_n = step.cost - g_n  # Cost (h(n))
        f_n = step.cost  # Total cost (f(n))
        print(f"Move: {move_str}, g(n): {g_n}, h(n): {h_n}, f(n): {f_n}")
        for i in range(0, len(step.board), 3):
            print(step.board[i:i+3])
        print()

initial = [2,8,3 ,1,6,4,7,0,5]
goal = [1,2,3,8,0,4,7,6,5]
result = a_star_search(initial, goal)

if result:
    print("Puzzle solved! Here's the sequence of steps:")
    print_solution(result)
else:
    print("No solution found.")


Puzzle solved! Here's the sequence of steps:
Solution path:
Move: Start, g(n): 0, h(n): 4, f(n): 4
[2, 8, 3]
[1, 6, 4]
[7, 0, 5]

Move: Up, g(n): 1, h(n): 3, f(n): 4
[2, 8, 3]
[1, 0, 4]
[7, 6, 5]

Move: Up, g(n): 2, h(n): 3, f(n): 5
[2, 0, 3]
[1, 8, 4]
[7, 6, 5]

Move: Left, g(n): 3, h(n): 2, f(n): 5
[0, 2, 3]
[1, 8, 4]
[7, 6, 5]

Move: Down, g(n): 4, h(n): 1, f(n): 5
[1, 2, 3]
[0, 8, 4]
[7, 6, 5]

Move: Right, g(n): 5, h(n): 0, f(n): 5
[1, 2, 3]
[8, 0, 4]
[7, 6, 5]

